# E11. 인공지능 작사가 만들기

# 11-7. 프로젝트: 멋진 작사가 만들기

## 1) 데이터 다운로드

## 2) 데이터 읽어오기

In [2]:
# tf
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [10]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해

import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/e/e11_lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)"]


## Step 3. 데이터 정제
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

---
`preprocess_sentence()` 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가면 잘라내기를 권합니다.

In [14]:
print(type(raw_corpus))
print(len(raw_corpus))
raw_corpus

<class 'list'>
187088


['Look, I was gonna go easy on you and not to hurt your feelings',
 "But I'm only going to get this one chance",
 "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)",
 "Just a feeling I've got, like something's about to happen, but I don't know what",
 "If that means, what I think it means, we're in trouble, big trouble,",
 "And if he is as bananas as you say, I'm not taking any chances",
 "You were just what the doctor ordered I'm beginning to feel like a Rap God, Rap God",
 'All my people from the front to the back nod, back nod',
 'Now who thinks their arms are long enough to slap box, slap box?',
 'They said I rap like a robot, so call me Rapbot But for me to rap like a computer must be in my genes',
 'I got a laptop in my back pocket',
 "My pen'll go off when I half-cock it",
 'Got a fat knot from that rap profit',
 'Made a living and a killing off it',
 'Ever since Bill Clinton was still in office',
 'With Monica Lewinsky feeling on his nut-sack',
 "I'm an MC 

In [19]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence


In [24]:
# 길이가 0인 문장 정제
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    
    corpus.append(preprocess_sentence(sentence))
    
print(corpus[:3])
len(corpus)

['<start> look , i was gonna go easy on you and not to hurt your feelings <end>', '<start> but i m only going to get this one chance <end>', '<start> something s wrong , i can feel it six minutes , slim shady , you re on <end>']


175986

In [21]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

175986

## Step 4. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!

---

`tokenize()` 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!

```python
enc_train, enc_val, dec_train, dec_val = <코드 작성>
```
여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.
```python
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
```
```python
out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요!

## Step 5. 인공지능 만들기

---

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

```python
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
```
데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

```python
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)
```

Q4. 모델이 생성한 가사 한 줄을 제출하세요.